In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
genre_df = spark.read.table('silver.movie_genre')
genre_df.show()

+-------+-------+
|movieId|genreId|
+-------+-------+
|      1|      6|
|      1|     11|
|      1|     21|
|      2|      7|
|      2|      9|
|      3|      9|
|      3|     15|
|      4|      7|
|      4|      9|
|      5|     15|
|      5|     18|
|      6|      3|
|      6|      9|
|      6|     14|
|      6|     27|
|      6|     28|
|      7|      6|
|      7|      9|
|      7|     21|
|      8|      9|
+-------+-------+
only showing top 20 rows



In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# Define the silver location
genre_goldLocation = "/mnt/datalake/gold/dim_movie_genre"

# Table name
genre_goldTable = "gold.Dim_Movie_Genre"

genre_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(genre_goldLocation)

In [0]:
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {genre_goldTable} USING delta LOCATION '{genre_goldLocation}'")

DataFrame[]

In [0]:
# Maintenance for Delta table
genre_DataDelta = DeltaTable.forName(spark, genre_goldTable)

if genre_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	genre_DataDelta.optimize()
	genre_DataDelta.vacuum()


In [0]:
%sql 
select * from gold.dim_Movie_Genre

movieId,genreId
1,6
1,11
1,21
2,7
2,9
3,9
3,15
4,7
4,9
5,15
